In [2]:
# Chapter 7.1 - LangChain 프롬프트 템플릿 (최신 버전 대응)
# 원서 코드의 LLMChain 및 chain.run() 메소드는 deprecated 되었습니다.
# 이 노트북은 최신 LangChain 버전(0.2.x+)에 맞게 수정된 코드입니다.

# 필요한 라이브러리 설치
# !pip install langchain==0.2.17 langchain-openai tiktoken

import os
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# =============================================================================
# 🔑 API 키 설정 (독자용)
# =============================================================================
import os
import getpass

print("OpenAI API 키 설정이 필요합니다.")
print("API 키 발급: https://platform.openai.com/api-keys")

# API 키가 이미 환경 변수에 있는지 확인
if 'OPENAI_API_KEY' not in os.environ:
    print("\n다음 중 한 가지 방법을 선택하세요:")
    print("1. 아래에서 직접 입력 (안전함 - 화면에 표시되지 않음)")
    print("2. 환경 변수로 설정 (export OPENAI_API_KEY=your-key)")
    print("3. .env 파일 사용")
    
    # 안전한 입력 방법
    try:
        api_key = getpass.getpass("\nAPI 키를 입력하세요 (입력 시 화면에 표시되지 않음): ")
        if api_key:
            os.environ['OPENAI_API_KEY'] = api_key
            print("✅ API 키가 안전하게 설정되었습니다!")
        else:
            print("⚠️ API 키가 입력되지 않았습니다.")
    except KeyboardInterrupt:
        print("\n입력이 취소되었습니다.")
else:
    print("✅ 기존 환경 변수의 API 키를 사용합니다.")

print("\n" + "="*60)

# =============================================================================
# LLM 모델 정의
# =============================================================================
llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0  # 일관된 결과를 위해 temperature를 0으로 설정
)

print("🤖 LLM 모델 초기화 완료")

# =============================================================================
# 프롬프트 템플릿 생성
# =============================================================================
template = """
Answer the question based on the context below.
If the question cannot be answered using the information provided, answer with "I don't know".

Context: Quantum computing is an emerging field that leverages quantum mechanics to solve
complex problems faster than classical computers.

Question: {query}
Answer:
"""

# 프롬프트 템플릿 객체 생성
prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

print("📝 프롬프트 템플릿 생성 완료")

# =============================================================================
# 방법 1: RunnableSequence 사용 (권장되는 최신 방법)
# =============================================================================
print("\n" + "="*60)
print("방법 1: RunnableSequence 사용 (LangChain 0.2+ 권장 방법)")
print("="*60)

# 최신 방식: prompt | llm 파이프라인
chain = prompt_template | llm

# 질문 준비
query = "What is the main advantage of quantum computing over classical computing?"

# 실행 (invoke 메소드 사용)
try:
    response = chain.invoke({"query": query})
    
    print(f"질문: {query}")
    print(f"답변: {response.content}")
    
except Exception as e:
    print(f"⚠️ API 키가 설정되지 않았거나 오류가 발생했습니다: {e}")
    print("💡 실제 실행을 위해서는 OpenAI API 키가 필요합니다.")

# =============================================================================
# 방법 2: 기존 LLMChain 방식과 비교 (참고용)
# =============================================================================
print("\n" + "="*60)
print("참고: 기존 방식 (Deprecated)")
print("="*60)

print("""
# 이전 방식 (더 이상 권장되지 않음):
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template)  # ⚠️ Deprecated
response = chain.run({"query": query})              # ⚠️ Deprecated

# 새로운 방식:
chain = prompt_template | llm                       # ✅ 권장
response = chain.invoke({"query": query})           # ✅ 권장
""")

# =============================================================================
# 추가 예제: 여러 질문 처리
# =============================================================================
print("\n" + "="*60)
print("추가 예제: 여러 질문 한 번에 처리")
print("="*60)

# 여러 질문 준비
questions = [
    "What is quantum computing?",
    "How does quantum computing work?",
    "What are the limitations of classical computers mentioned?",
    "Can quantum computers replace smartphones?"  # 컨텍스트에 없는 정보
]

# 배치 처리
try:
    # batch 메소드로 여러 질문을 한 번에 처리
    batch_inputs = [{"query": q} for q in questions]
    responses = chain.batch(batch_inputs)
    
    for i, (question, response) in enumerate(zip(questions, responses), 1):
        print(f"\n질문 {i}: {question}")
        print(f"답변 {i}: {response.content}")
        print("-" * 40)
        
except Exception as e:
    print(f"⚠️ 배치 처리 중 오류 발생: {e}")

# =============================================================================
# 한국어 프롬프트 예제
# =============================================================================
print("\n" + "="*60)
print("한국어 프롬프트 예제")
print("="*60)

# 한국어 프롬프트 템플릿
korean_template = """
아래 맥락을 바탕으로 질문에 답하세요.
제공된 정보로 답할 수 없는 질문이라면 "잘 모르겠습니다"라고 답하세요.

맥락: 양자 컴퓨팅은 양자역학을 활용하여 기존 컴퓨터보다 복잡한 문제를 더 빠르게 해결하는 
새로운 분야입니다.

질문: {query}
답변:
"""

korean_prompt = PromptTemplate(
    input_variables=["query"],
    template=korean_template
)

# 한국어 체인 생성
korean_chain = korean_prompt | llm

korean_question = "양자 컴퓨팅의 주요 장점은 무엇인가요?"

try:
    korean_response = korean_chain.invoke({"query": korean_question})
    print(f"한국어 질문: {korean_question}")
    print(f"한국어 답변: {korean_response.content}")
    
except Exception as e:
    print(f"⚠️ 한국어 예제 실행 중 오류: {e}")

# =============================================================================
# 성능 및 디버깅 팁
# =============================================================================
print("\n" + "="*60)
print("💡 성능 및 디버깅 팁")
print("="*60)

print("""
1. 스트리밍 응답 사용:
   for chunk in chain.stream({"query": "your question"}):
       print(chunk.content, end="", flush=True)

2. 비동기 처리:
   response = await chain.ainvoke({"query": "your question"})

3. 프롬프트 디버깅:
   formatted_prompt = prompt_template.format(query="your question")
   print(formatted_prompt)

4. 메타데이터 활용:
   response = chain.invoke(
       {"query": "your question"},
       config={"metadata": {"user_id": "12345"}}
   )
""")

print("\n🎉 Chapter 7.1 프롬프트 템플릿 예제 완료!")
print("📚 다음 단계: 7.2절 퓨샷 프롬프트와 예시 선택기")

OpenAI API 키 설정이 필요합니다.
API 키 발급: https://platform.openai.com/api-keys
✅ 기존 환경 변수의 API 키를 사용합니다.

🤖 LLM 모델 초기화 완료
📝 프롬프트 템플릿 생성 완료

방법 1: RunnableSequence 사용 (LangChain 0.2+ 권장 방법)
질문: What is the main advantage of quantum computing over classical computing?
답변: The main advantage of quantum computing over classical computing is its ability to solve complex problems faster due to leveraging quantum mechanics.

참고: 기존 방식 (Deprecated)

# 이전 방식 (더 이상 권장되지 않음):
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template)  # ⚠️ Deprecated
response = chain.run({"query": query})              # ⚠️ Deprecated

# 새로운 방식:
chain = prompt_template | llm                       # ✅ 권장
response = chain.invoke({"query": query})           # ✅ 권장


추가 예제: 여러 질문 한 번에 처리

질문 1: What is quantum computing?
답변 1: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.
----------------------------------------
